In [1]:
import sys
import pygame

from PyQt5 import QtCore, QtGui, QtWidgets
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import librosa.display
import tensorflow as tf
from keras.models import load_model
from pydub import AudioSegment
from playsound import playsound
import os

model = tf.keras.models.load_model('snoring_local.h5')
global index
global file_path

folder_path = 'snoringds/full_test'
file_list = os.listdir(folder_path)


def preprocess_audio(file_path):
    y, sr = librosa.load(file_path)
    print(file_path)
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    spectrogram = np.expand_dims(spectrogram, axis=-1)
    return spectrogram


class PlotViewer(QtWidgets.QMainWindow):
    def __init__(self):
        super(PlotViewer, self).__init__()
        fig = plt.Figure(figsize=(6, 4), dpi=100, facecolor="#3533CD")

        # Set the window dimensions
        window_width = 800
        window_height = 900
        screen = QtWidgets.QApplication.primaryScreen()
        self.setWindowIcon(QtGui.QIcon("medical-logo.jpg"))
        screen_rect = screen.availableGeometry()
        window_x = int((screen_rect.width() - window_width) / 2)
        window_y = int((screen_rect.height() - window_height) / 2)
        self.setGeometry(window_x, window_y, window_width, window_height)

        # Create the header label
        header_label = QtWidgets.QLabel("Plot Viewer", self)
        header_label.setFont(QtGui.QFont("Arial", 18))
        header_label.setStyleSheet("background-color:#3533CD; color: white;")  # Set font color to white
        header_label.setAlignment(QtCore.Qt.AlignCenter)
        header_label.setGeometry(10, 10, window_width - 20, 30)

        # Create the input space
        input_frame = QtWidgets.QFrame(self)
        input_frame.setStyleSheet("background-color: #3533CD;")
        input_frame.setGeometry(60, 40, 700, 270)

        input_label = QtWidgets.QLabel("Enter Index:", input_frame)
        input_label.setFont(QtGui.QFont("Arial", 12))
        input_label.setStyleSheet("background-color: #3533CD; color: white;")  # Set font color to white
        input_label.setGeometry(220, 20, 100, 50)  # Adjust the position of the input label

        self.input_space = QtWidgets.QLineEdit(input_frame)
        self.input_space.setFont(QtGui.QFont("Arial", 12))
        self.input_space.setGeometry(340, 30, 100, 30)
        self.input_space.setStyleSheet("background-color: #3533CD; color: white;") 

        # Create the button frame
        button_frame = QtWidgets.QFrame(self)
        button_frame.setStyleSheet("background-color:#3533CD;")
        button_frame.setGeometry(int((window_width - 560) / 2), 100, 560, 600)

        button_spacing = 70
        button_size = 50
        button_margin = int((500 - (button_spacing * 4 + button_size)) / 2)

        play_button = QtWidgets.QPushButton("", button_frame)
        play_button.setIcon(QtGui.QIcon("play.png"))
        play_button.setIconSize(QtCore.QSize(button_size, button_size))
        play_button.setStyleSheet("background-color: transparent;")
        play_button.setGeometry(button_margin, 20, button_size, button_size)
        play_button.clicked.connect(self.play_audio)

        plot_button = QtWidgets.QPushButton("", button_frame)
        plot_button.setIcon(QtGui.QIcon("plotf.png"))
        plot_button.setIconSize(QtCore.QSize(button_size, button_size))
        plot_button.setStyleSheet("background-color: transparent;")
        plot_button.setGeometry(button_margin + button_spacing, 20, button_size, button_size)
        plot_button.clicked.connect(self.plot_graph)

        spec = QtWidgets.QPushButton("", button_frame)
        spec.setIcon(QtGui.QIcon("spec.png"))
        spec.setIconSize(QtCore.QSize(button_size, button_size))
        spec.setStyleSheet("background-color: transparent;")
        spec.setGeometry(button_margin + button_spacing * 2, 20, button_size, button_size)
        spec.clicked.connect(self.show_spectrogram)

        save_button = QtWidgets.QPushButton("", button_frame)
        save_button.setIcon(QtGui.QIcon("brain.png"))
        save_button.setIconSize(QtCore.QSize(button_size, button_size))
        save_button.setStyleSheet("background-color: transparent;")
        save_button.setGeometry(button_margin + button_spacing * 3, 20, button_size, button_size)
        save_button.clicked.connect(self.save_plot)

        refresh_button = QtWidgets.QPushButton("", button_frame)
        refresh_button.setIcon(QtGui.QIcon("bv.png"))
        refresh_button.setIconSize(QtCore.QSize(button_size, button_size))
        refresh_button.setStyleSheet("background-color: transparent;")
        refresh_button.setGeometry(button_margin + button_spacing * 4, 20, button_size, button_size)
        refresh_button.clicked.connect(self.refresh)
        self.index = 0

        # ... Existing code ...

        # Create the subtract button
        subtract_button = QtWidgets.QPushButton("-", input_frame)
        subtract_button.setFont(QtGui.QFont("Arial", 12))
        subtract_button.setStyleSheet("background-color: #3533CD; color: white;")
        subtract_button.setGeometry(450, 30, 30, 30)
        subtract_button.clicked.connect(self.subtract_index)

        # Create the add button
        add_button = QtWidgets.QPushButton("+", input_frame)
        add_button.setFont(QtGui.QFont("Arial", 12))
        add_button.setStyleSheet("background-color: #3533CD; color: white;")  # Set font color to white
        add_button.setGeometry(480, 30, 30, 30)
        add_button.clicked.connect(self.add_index)

        exit_button = QtWidgets.QPushButton("", button_frame)
        exit_button.setIcon(QtGui.QIcon("exit.png"))
        exit_button.setIconSize(QtCore.QSize(button_size, button_size))
        exit_button.setStyleSheet("background-color: transparent;")
        exit_button.setGeometry(button_margin + button_spacing * 5, 20, button_size, button_size)
        exit_button.clicked.connect(self.exit_application)

        # Create the plot space
        self.plot_space = QtWidgets.QWidget(self)
        self.plot_space.setStyleSheet("background-color:#3533CD")
        self.plot_space.setGeometry(10, 250, window_width - 20, 400)

        # Create the beat label
        self.beat_label = QtWidgets.QLabel(self)
        self.beat_label.setFont(QtGui.QFont("Arial", 12))
        self.beat_label.setStyleSheet("background-color:#3533CD; color: white;")  # Set font color to white
        self.beat_label.setGeometry(int((window_width -700) / 2), 660, window_width - 100, 30)

        # Create the footer label
        footer_label = QtWidgets.QLabel("© 2023 AWAKE Plot Viewer. All rights reserved.", self)
        footer_label.setFont(QtGui.QFont("Arial", 10))
        footer_label.setStyleSheet("background-color:#3533CD; color: white;")  # Set font color to white
        footer_label.setAlignment(QtCore.Qt.AlignCenter)
        footer_label.setGeometry(10, 790, window_width - 20, 20)

        plot = fig.add_subplot(111)
        plot.plot(0, 0)
        plot.set_xlabel('X', color='white')  # Set X-axis label color to white
        plot.set_ylabel('Y', color='white')  # Set Y-axis label color to white
        plot.set_title('Plot', color='white')  # Set title color to white

        # Set tick color and tick label color to white
        plot.tick_params(colors='white')
        plot.xaxis.label.set_color('white')
        plot.yaxis.label.set_color('white')

         
        canvas = FigureCanvas(fig)
        canvas.setGeometry(0, 0, 780, 400)
        canvas.setStyleSheet("background-color:#3533CD;")
        canvas.setParent(self.plot_space)

        self.setWindowTitle("Plot Viewer")
        self.setStyleSheet("background-color:#3533CD;")

    def refresh(self):
        for child in self.plot_space.children():
            child.deleteLater()
        self.input_space.clear()
        self.beat_label.setText("")
        
    def add_index(self):
        index_text = self.input_space.text()
        if index_text:
            self.index = int(index_text) + 1
            self.input_space.setText(str(self.index))

    def subtract_index(self):
        index_text = self.input_space.text()
        if index_text and int(index_text) > 0:
            self.index = int(index_text) - 1
            self.input_space.setText(str(self.index))

    def plot_graph(self):
        for child in self.plot_space.children():
            child.deleteLater()

        index = int(self.input_space.text())

        if 0 <= index < len(file_list):
            file_name = file_list[index]
            file_path = os.path.join(folder_path, file_name)
            audio, sr = librosa.load(file_path)
            fig = plt.Figure(figsize=(6, 4), dpi=100, facecolor="#3533CD")
            plot = fig.add_subplot(111)
            plot.plot(audio)
            plot.set_xlabel('X', color='white')  # Set X-axis label color to white
            plot.set_ylabel('Y', color='white')  # Set Y-axis label color to white
            plot.set_title('Plot', color='white')  # Set title color to white

            # Set tick color and tick label color to white
            plot.tick_params(colors='white')
            plot.xaxis.label.set_color('white')
            plot.yaxis.label.set_color('white')

            # Set the color of the frame and the spines to white
            plot.spines['top'].set_color('white')
            plot.spines['bottom'].set_color('white')
            plot.spines['left'].set_color('white')
            plot.spines['right'].set_color('white')

            canvas = FigureCanvas(fig)
            canvas.setGeometry(0, 0, 780, 400)
            canvas.setStyleSheet("background-color:#3533CD;")
            canvas.setParent(self.plot_space)
            canvas.show()


    def save_plot(self):
        index = int(self.input_space.text())

        if 0 <= index < len(file_list):
            file_name = file_list[index]
            file_path = os.path.join(folder_path, file_name)
            new_spectrogram = preprocess_audio(file_path)
            prediction = model.predict(np.array([new_spectrogram]))
            if prediction[0][0] > 0.1:
                prediction_text = 'The audio file contains snoring.'
            else:
                prediction_text = 'The audio file does not contain snoring.'

            self.beat_label.setText(prediction_text)
            self.beat_label.setAlignment(QtCore.Qt.AlignCenter)
            self.beat_label.setStyleSheet("background-color: black; color: white;")


    def play_audio(self):
        index = int(self.input_space.text())

        if 0 <= index < len(file_list):
            file_name = file_list[index]
            file_path = os.path.join(folder_path, file_name)
            string_with_backslashes = file_path
            string_with_forwardslashes = string_with_backslashes.replace('\\', '/')
            print(string_with_forwardslashes)

            #playsound(string_with_forwardslashes, alias='my_audio_alias')
            pygame.mixer.init()
            pygame.mixer.music.load(string_with_forwardslashes)
            pygame.mixer.music.play()

    def show_spectrogram(self):
        for child in self.plot_space.children():
            child.deleteLater()

        index = int(self.input_space.text())

        if 0 <= index < len(file_list):
            file_name = file_list[index]
            file_path = os.path.join(folder_path, file_name)
            audio, sr = librosa.load(file_path)
            spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr)
            fig = plt.Figure(figsize=(6, 4), dpi=100, facecolor="#3533CD")
            plot = fig.add_subplot(111)
            img = librosa.display.specshow(librosa.power_to_db(spectrogram, ref=np.max), x_axis='time', y_axis='mel',
                                           sr=sr, ax=plot)
            fig.colorbar(img, ax=plot, format='%+2.0f dB')

            # Set axis label color and font color to white
            plot.set_title('Spectrogram', color='white')
            plot.set_xlabel('Time', color='white')
            plot.set_ylabel('Frequency', color='white')

            # Set tick color and tick label color to white
            plot.tick_params(colors='white')
            plot.xaxis.label.set_color('white')
            plot.yaxis.label.set_color('white')

            # Set the color of the frame and the spines to white
            plot.spines['top'].set_color('white')
            plot.spines['bottom'].set_color('white')
            plot.spines['left'].set_color('white')
            plot.spines['right'].set_color('white')

            canvas = FigureCanvas(fig)
            canvas.setGeometry(0, 0, 780, 400)
            canvas.setStyleSheet("background-color:#3533CD;")
            canvas.setParent(self.plot_space)
            canvas.show()


    def exit_application(self):
        sys.exit()


app = QtWidgets.QApplication(sys.argv)
viewer = PlotViewer()

viewer.show()
sys.exit(app.exec_())


pygame 2.4.0 (SDL 2.26.4, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\Adithya\anaconda3\envs\projects\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


snoringds/full_test/0_108.wav
snoringds/full_test/0_128.wav
snoringds/full_test\0_128.wav
1/1 [==============================] - 0s 224ms/step
snoringds/full_test/0_199.wav
snoringds/full_test\0_199.wav
1/1 [==============================] - 0s 23ms/step
snoringds/full_test/0_191.wav
snoringds/full_test/F_16.wav
snoringds/full_test/F_16.wav
snoringds/full_test/F_16.wav
snoringds/full_test/F_160.wav
snoringds/full_test\F_160.wav
1/1 [==============================] - 0s 24ms/step
snoringds/full_test/F_162.wav
snoringds/full_test\F_162.wav
1/1 [==============================] - 0s 20ms/step
snoringds/full_test/0_168.wav
snoringds/full_test\0_168.wav
1/1 [==============================] - 0s 23ms/step


SystemExit: 0

C:\Users\Adithya\anaconda3\envs\projects\lib\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# !pip install pygame

In [ ]:
import sys


from PyQt5 import QtCore, QtGui, QtWidgets
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import librosa.display
import tensorflow as tf
from keras.models import load_model
from pydub import AudioSegment
from playsound import playsound
import os

model = tf.keras.models.load_model('best_model.h5')
global index
global file_path

 
test_df = pd.read_csv('mitbih_test.csv', header=None)

def preprocess_audio(file_path):
    y, sr = librosa.load(file_path)
    print(file_path)
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    spectrogram = np.expand_dims(spectrogram, axis=-1)
    return spectrogram


class PlotViewer(QtWidgets.QMainWindow):
    def __init__(self):
        super(PlotViewer, self).__init__()
        fig = plt.Figure(figsize=(6, 4), dpi=100, facecolor="#3533CD")

        # Set the window dimensions
        window_width = 800
        window_height = 900
        screen = QtWidgets.QApplication.primaryScreen()
        self.setWindowIcon(QtGui.QIcon("medical-logo.jpg"))
        screen_rect = screen.availableGeometry()
        window_x = int((screen_rect.width() - window_width) / 2)
        window_y = int((screen_rect.height() - window_height) / 2)
        self.setGeometry(window_x, window_y, window_width, window_height)

        # Create the header label
        header_label = QtWidgets.QLabel("Plot Viewer", self)
        header_label.setFont(QtGui.QFont("Arial", 18))
        header_label.setStyleSheet("background-color:#3533CD; color: white;")  # Set font color to white
        header_label.setAlignment(QtCore.Qt.AlignCenter)
        header_label.setGeometry(10, 10, window_width - 20, 30)

        # Create the input space
        input_frame = QtWidgets.QFrame(self)
        input_frame.setStyleSheet("background-color: #3533CD;")
        input_frame.setGeometry(60, 40, 700, 270)

        input_label = QtWidgets.QLabel("Enter Index:", input_frame)
        input_label.setFont(QtGui.QFont("Arial", 12))
        input_label.setStyleSheet("background-color: #3533CD; color: white;")  # Set font color to white
        input_label.setGeometry(220, 20, 100, 50)  # Adjust the position of the input label

        self.input_space = QtWidgets.QLineEdit(input_frame)
        self.input_space.setFont(QtGui.QFont("Arial", 12))
        self.input_space.setGeometry(340, 30, 100, 30)
        self.input_space.setStyleSheet("background-color: #3533CD; color: white;") 

        # Create the button frame
        button_frame = QtWidgets.QFrame(self)
        button_frame.setStyleSheet("background-color:#3533CD;")
        button_frame.setGeometry(int((window_width - 560) / 2), 100, 560, 600)

        button_spacing = 70
        button_size = 50
        button_margin = int((500 - (button_spacing * 4 + button_size)) / 2)

        

        plot_button = QtWidgets.QPushButton("", button_frame)
        plot_button.setIcon(QtGui.QIcon("plotf.png"))
        plot_button.setIconSize(QtCore.QSize(button_size, button_size))
        plot_button.setStyleSheet("background-color: transparent;")
        plot_button.setGeometry(button_margin + button_spacing, 20, button_size, button_size)
        plot_button.clicked.connect(self.plot_graph)

        spec = QtWidgets.QPushButton("", button_frame)
        spec.setIcon(QtGui.QIcon("spec.png"))
        spec.setIconSize(QtCore.QSize(button_size, button_size))
        spec.setStyleSheet("background-color: transparent;")
        spec.setGeometry(button_margin + button_spacing * 2, 20, button_size, button_size)
        spec.clicked.connect(self.show_spectrogram)

        save_button = QtWidgets.QPushButton("", button_frame)
        save_button.setIcon(QtGui.QIcon("brain.png"))
        save_button.setIconSize(QtCore.QSize(button_size, button_size))
        save_button.setStyleSheet("background-color: transparent;")
        save_button.setGeometry(button_margin + button_spacing * 3, 20, button_size, button_size)
        save_button.clicked.connect(self.save_plot)

        refresh_button = QtWidgets.QPushButton("", button_frame)
        refresh_button.setIcon(QtGui.QIcon("bv.png"))
        refresh_button.setIconSize(QtCore.QSize(button_size, button_size))
        refresh_button.setStyleSheet("background-color: transparent;")
        refresh_button.setGeometry(button_margin + button_spacing * 4, 20, button_size, button_size)
        refresh_button.clicked.connect(self.refresh)
        self.index = 0

        # ... Existing code ...

        # Create the subtract button
        subtract_button = QtWidgets.QPushButton("-", input_frame)
        subtract_button.setFont(QtGui.QFont("Arial", 12))
        subtract_button.setStyleSheet("background-color: #3533CD; color: white;")
        subtract_button.setGeometry(450, 30, 30, 30)
        subtract_button.clicked.connect(self.subtract_index)

        # Create the add button
        add_button = QtWidgets.QPushButton("+", input_frame)
        add_button.setFont(QtGui.QFont("Arial", 12))
        add_button.setStyleSheet("background-color: #3533CD; color: white;")  # Set font color to white
        add_button.setGeometry(480, 30, 30, 30)
        add_button.clicked.connect(self.add_index)

        exit_button = QtWidgets.QPushButton("", button_frame)
        exit_button.setIcon(QtGui.QIcon("exit.png"))
        exit_button.setIconSize(QtCore.QSize(button_size, button_size))
        exit_button.setStyleSheet("background-color: transparent;")
        exit_button.setGeometry(button_margin + button_spacing * 5, 20, button_size, button_size)
        exit_button.clicked.connect(self.exit_application)

        # Create the plot space
        self.plot_space = QtWidgets.QWidget(self)
        self.plot_space.setStyleSheet("background-color:#3533CD")
        self.plot_space.setGeometry(10, 250, window_width - 20, 400)

        # Create the beat label
        self.beat_label = QtWidgets.QLabel(self)
        self.beat_label.setFont(QtGui.QFont("Arial", 12))
        self.beat_label.setStyleSheet("background-color:#3533CD; color: white;")  # Set font color to white
        self.beat_label.setGeometry(int((window_width -700) / 2), 660, window_width - 100, 30)

        # Create the footer label
        footer_label = QtWidgets.QLabel("© 2023 AWAKE Plot Viewer. All rights reserved.", self)
        footer_label.setFont(QtGui.QFont("Arial", 10))
        footer_label.setStyleSheet("background-color:#3533CD; color: white;")  # Set font color to white
        footer_label.setAlignment(QtCore.Qt.AlignCenter)
        footer_label.setGeometry(10, 790, window_width - 20, 20)

        plot = fig.add_subplot(111)
        plot.plot(0, 0)
        plot.set_xlabel('X', color='white')  # Set X-axis label color to white
        plot.set_ylabel('Y', color='white')  # Set Y-axis label color to white
        plot.set_title('Plot', color='white')  # Set title color to white

        # Set tick color and tick label color to white
        plot.tick_params(colors='white')
        plot.xaxis.label.set_color('white')
        plot.yaxis.label.set_color('white')

         
        canvas = FigureCanvas(fig)
        canvas.setGeometry(0, 0, 780, 400)
        canvas.setStyleSheet("background-color:#3533CD;")
        canvas.setParent(self.plot_space)

        self.setWindowTitle("Plot Viewer")
        self.setStyleSheet("background-color:#3533CD;")

    def refresh(self):
        for child in self.plot_space.children():
            child.deleteLater()
        self.input_space.clear()
        self.beat_label.setText("")
        
    def add_index(self):
        index_text = self.input_space.text()
        if index_text:
            self.index = int(index_text) + 1
            self.input_space.setText(str(self.index))

    def subtract_index(self):
        index_text = self.input_space.text()
        if index_text and int(index_text) > 0:
            self.index = int(index_text) - 1
            self.input_space.setText(str(self.index))

    def plot_graph(self):
        for child in self.plot_space.children():
            child.deleteLater()

        index = int(self.input_space.text())
        fig = plt.Figure(figsize=(6, 4), dpi=100, facecolor="#3533CD")
        plot = fig.add_subplot(111)
        plot.plot(test_df.iloc[index].values)
        plot.set_xlabel('X', color='white')  # Set X-axis label color to white
        plot.set_ylabel('Y', color='white')  # Set Y-axis label color to white
        plot.set_title('Plot', color='white')  # Set title color to white

        # Set tick color and tick label color to white
        plot.tick_params(colors='white')
        plot.xaxis.label.set_color('white')
        plot.yaxis.label.set_color('white')

        # Set the color of the frame and the spines to white
        plot.spines['top'].set_color('white')
        plot.spines['bottom'].set_color('white')
        plot.spines['left'].set_color('white')
        plot.spines['right'].set_color('white')

        canvas = FigureCanvas(fig)
        canvas.setGeometry(0, 0, 780, 400)
        canvas.setStyleSheet("background-color:#3533CD;")
        canvas.setParent(self.plot_space)
        canvas.show()

      

    # Retrieve the array element from test_df based on the input index
     


     
            


    def save_plot(self):
        index = int(self.input_space.text())

        x_array = test_df.iloc[index].values
        x_reshaped = x_array[:186].reshape(-1, 186, 1)
        y_predict = model.predict(x_reshaped)
        y_pred_inv = np.argmax(y_predict, axis=1)

        if 0 in y_pred_inv:
            beat_type = "Non-ectopic beats detected"
        elif 1 in y_pred_inv:
            beat_type = "Supraventricular ectopic beats detected"
        elif 2 in y_pred_inv:
            beat_type = "Ventricular ectopic beats detected"
        elif 3 in y_pred_inv:
            beat_type = "Fusion Beats detected"
        elif 4 in y_pred_inv:
            beat_type = "Unknown Beats detected"
        else:
            beat_type = "Beat type not detected"
        self.beat_label.setText(beat_type)
        self.beat_label.setAlignment(QtCore.Qt.AlignCenter)
        self.beat_label.setStyleSheet("background-color: black; color: white;")


    def play_audio(self):
        index = int(self.input_space.text())

        if 0 <= index < len(file_list):
            file_name = file_list[index]
            file_path = os.path.join(folder_path, file_name)
            string_with_backslashes = file_path
            string_with_forwardslashes = string_with_backslashes.replace('\\', '/')
            print(string_with_forwardslashes)

            #playsound(string_with_forwardslashes, alias='my_audio_alias')
            pygame.mixer.init()
            pygame.mixer.music.load(string_with_forwardslashes)
            pygame.mixer.music.play()

   
    def show_spectrogram(self):
        class_number=0
        size=70
        min_=5
        bins=65
        for child in self.plot_space.children():
            child.deleteLater()
            
            

        

        index = int(self.input_space.text())

        img = test_df.iloc[20000].values
        
        
        ###
        img=test_df.loc[test_df[187]==class_number].values
        img=img[:,min_:size]
        img_flatten=img.flatten()

        final1=np.arange(min_,size)
        for i in range (img.shape[0]-1):
            tempo1=np.arange(min_,size)
            final1=np.concatenate((final1, tempo1), axis=None)
     
        img=plt.hist(final1,img_flatten, cmap=plt.cm.jet)
        plt.show()
    
 

        """spectrogram = librosa.feature.melspectrogram(y=img, sr=0.1, n_fft=100, hop_length=256)
        fig = plt.Figure(figsize=(6, 4), dpi=100, facecolor="#3533CD")
        plot = fig.add_subplot(111)
        img = librosa.display.specshow(spectrogram,  x_axis='time', y_axis='mel',
                                       sr=0.1, ax=plot)"""
        fig = plt.Figure(figsize=(6, 4), dpi=100, facecolor="#3533CD")
        plot = fig.add_subplot(111)
 
        # Set axis label color and font color to white
        plot.set_title('Spectrogram', color='white')
        plot.set_xlabel('Time', color='white')
        plot.set_ylabel('Frequency', color='white')

        # Set tick color and tick label color to white
        plot.tick_params(colors='white')
        plot.xaxis.label.set_color('white')
        plot.yaxis.label.set_color('white')

        # Set the color of the frame and the spines to white
        plot.spines['top'].set_color('white')
        plot.spines['bottom'].set_color('white')
        plot.spines['left'].set_color('white')
        plot.spines['right'].set_color('white')

        canvas = FigureCanvas(img)
        canvas.setGeometry(0, 0, 780, 400)
        canvas.setStyleSheet("background-color:#3533CD;")
        canvas.setParent(self.plot_space)
        canvas.show()




    def exit_application(self):
        sys.exit()


app = QtWidgets.QApplication(sys.argv)
viewer = PlotViewer()


viewer.show()
sys.exit(app.exec_())


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QVBoxLayout, QWidget

class CodeWindow(QMainWindow):
    def __init__(self, code):
        super().__init__()
        self.setWindowTitle("Code Window")
        self.setGeometry(100, 100, 400, 300)
        
        self.code = code

    def run_code(self):
        exec(self.code)

class MenuWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Menu Window")
        self.setGeometry(100, 100, 200, 150)
        
        layout = QVBoxLayout()
        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)
        
        button1 = QPushButton("Code 1", self)
        button1.clicked.connect(self.open_code_window1)
        layout.addWidget(button1)

        button2 = QPushButton("Code 2", self)
        button2.clicked.connect(self.open_code_window2)
        layout.addWidget(button2)
        
    def open_code_window1(self):
        code = """
print("This is Code 1")
"""
        code_window = CodeWindow(code)
        code_window.show()

    def open_code_window2(self):
        code = """
print("This is Code 2")
"""
        code_window = CodeWindow(code)
        code_window.show()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    menu_window = MenuWindow()
    menu_window.show()
    sys.exit(app.exec_())


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QVBoxLayout, QWidget, QMessageBox

class CodeWindow(QMainWindow):
    def __init__(self, code):
        super().__init__()
        self.setWindowTitle("Code Window")
        self.setGeometry(100, 100, 400, 300)
        
        self.code = code

    def run_code(self):
        try:
            exec(self.code)
        except Exception as e:
            QMessageBox.critical(self, "Error", str(e))

class MenuWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Menu Window")
        self.setGeometry(100, 100, 200, 150)
        
        layout = QVBoxLayout()
        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)
        
        button1 = QPushButton("Code 1", self)
        button1.clicked.connect(self.open_code_window1)
        layout.addWidget(button1)

        button2 = QPushButton("Code 2", self)
        button2.clicked.connect(self.open_code_window2)
        layout.addWidget(button2)
        
    def open_code_window1(self):
        code = """
print("This is Code 1")
"""
        code_window = CodeWindow(code)
        code_window.show()
        code_window.run_code()

    def open_code_window2(self):
        import sys


        from PyQt5 import QtCore, QtGui, QtWidgets
        from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
        import matplotlib.pyplot as plt
        import pandas as pd
        import numpy as np
        import librosa.display
        import tensorflow as tf
        from keras.models import load_model
        from pydub import AudioSegment
        from playsound import playsound
        import os

        model = tf.keras.models.load_model('best_model.h5')
        global index
        global file_path


        test_df = pd.read_csv('mitbih_test.csv', header=None)

        def preprocess_audio(file_path):
            y, sr = librosa.load(file_path)
            print(file_path)
            spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
            spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
            spectrogram = np.expand_dims(spectrogram, axis=-1)
            return spectrogram


        class PlotViewer(QtWidgets.QMainWindow):
            def __init__(self):
                super(PlotViewer, self).__init__()
                fig = plt.Figure(figsize=(6, 4), dpi=100, facecolor="#3533CD")

                # Set the window dimensions
                window_width = 800
                window_height = 900
                screen = QtWidgets.QApplication.primaryScreen()
                self.setWindowIcon(QtGui.QIcon("medical-logo.jpg"))
                screen_rect = screen.availableGeometry()
                window_x = int((screen_rect.width() - window_width) / 2)
                window_y = int((screen_rect.height() - window_height) / 2)
                self.setGeometry(window_x, window_y, window_width, window_height)

                # Create the header label
                header_label = QtWidgets.QLabel("Plot Viewer", self)
                header_label.setFont(QtGui.QFont("Arial", 18))
                header_label.setStyleSheet("background-color:#3533CD; color: white;")  # Set font color to white
                header_label.setAlignment(QtCore.Qt.AlignCenter)
                header_label.setGeometry(10, 10, window_width - 20, 30)

                # Create the input space
                input_frame = QtWidgets.QFrame(self)
                input_frame.setStyleSheet("background-color: #3533CD;")
                input_frame.setGeometry(60, 40, 700, 270)

                input_label = QtWidgets.QLabel("Enter Index:", input_frame)
                input_label.setFont(QtGui.QFont("Arial", 12))
                input_label.setStyleSheet("background-color: #3533CD; color: white;")  # Set font color to white
                input_label.setGeometry(220, 20, 100, 50)  # Adjust the position of the input label

                self.input_space = QtWidgets.QLineEdit(input_frame)
                self.input_space.setFont(QtGui.QFont("Arial", 12))
                self.input_space.setGeometry(340, 30, 100, 30)
                self.input_space.setStyleSheet("background-color: #3533CD; color: white;") 

                # Create the button frame
                button_frame = QtWidgets.QFrame(self)
                button_frame.setStyleSheet("background-color:#3533CD;")
                button_frame.setGeometry(int((window_width - 560) / 2), 100, 560, 600)

                button_spacing = 70
                button_size = 50
                button_margin = int((500 - (button_spacing * 4 + button_size)) / 2)



                plot_button = QtWidgets.QPushButton("", button_frame)
                plot_button.setIcon(QtGui.QIcon("plotf.png"))
                plot_button.setIconSize(QtCore.QSize(button_size, button_size))
                plot_button.setStyleSheet("background-color: transparent;")
                plot_button.setGeometry(button_margin + button_spacing, 20, button_size, button_size)
                plot_button.clicked.connect(self.plot_graph)

                spec = QtWidgets.QPushButton("", button_frame)
                spec.setIcon(QtGui.QIcon("spec.png"))
                spec.setIconSize(QtCore.QSize(button_size, button_size))
                spec.setStyleSheet("background-color: transparent;")
                spec.setGeometry(button_margin + button_spacing * 2, 20, button_size, button_size)
                spec.clicked.connect(self.show_spectrogram)

                save_button = QtWidgets.QPushButton("", button_frame)
                save_button.setIcon(QtGui.QIcon("brain.png"))
                save_button.setIconSize(QtCore.QSize(button_size, button_size))
                save_button.setStyleSheet("background-color: transparent;")
                save_button.setGeometry(button_margin + button_spacing * 3, 20, button_size, button_size)
                save_button.clicked.connect(self.save_plot)

                refresh_button = QtWidgets.QPushButton("", button_frame)
                refresh_button.setIcon(QtGui.QIcon("bv.png"))
                refresh_button.setIconSize(QtCore.QSize(button_size, button_size))
                refresh_button.setStyleSheet("background-color: transparent;")
                refresh_button.setGeometry(button_margin + button_spacing * 4, 20, button_size, button_size)
                refresh_button.clicked.connect(self.refresh)
                self.index = 0

                # ... Existing code ...

                # Create the subtract button
                subtract_button = QtWidgets.QPushButton("-", input_frame)
                subtract_button.setFont(QtGui.QFont("Arial", 12))
                subtract_button.setStyleSheet("background-color: #3533CD; color: white;")
                subtract_button.setGeometry(450, 30, 30, 30)
                subtract_button.clicked.connect(self.subtract_index)

                # Create the add button
                add_button = QtWidgets.QPushButton("+", input_frame)
                add_button.setFont(QtGui.QFont("Arial", 12))
                add_button.setStyleSheet("background-color: #3533CD; color: white;")  # Set font color to white
                add_button.setGeometry(480, 30, 30, 30)
                add_button.clicked.connect(self.add_index)

                exit_button = QtWidgets.QPushButton("", button_frame)
                exit_button.setIcon(QtGui.QIcon("exit.png"))
                exit_button.setIconSize(QtCore.QSize(button_size, button_size))
                exit_button.setStyleSheet("background-color: transparent;")
                exit_button.setGeometry(button_margin + button_spacing * 5, 20, button_size, button_size)
                #exit_button.clicked.connect(self.exit_application)

                # Create the plot space
                self.plot_space = QtWidgets.QWidget(self)
                self.plot_space.setStyleSheet("background-color:#3533CD")
                self.plot_space.setGeometry(10, 250, window_width - 20, 400)

                # Create the beat label
                self.beat_label = QtWidgets.QLabel(self)
                self.beat_label.setFont(QtGui.QFont("Arial", 12))
                self.beat_label.setStyleSheet("background-color:#3533CD; color: white;")  # Set font color to white
                self.beat_label.setGeometry(int((window_width -700) / 2), 660, window_width - 100, 30)

                # Create the footer label
                footer_label = QtWidgets.QLabel("© 2023 AWAKE Plot Viewer. All rights reserved.", self)
                footer_label.setFont(QtGui.QFont("Arial", 10))
                footer_label.setStyleSheet("background-color:#3533CD; color: white;")  # Set font color to white
                footer_label.setAlignment(QtCore.Qt.AlignCenter)
                footer_label.setGeometry(10, 790, window_width - 20, 20)

                plot = fig.add_subplot(111)
                plot.plot(0, 0)
                plot.set_xlabel('X', color='white')  # Set X-axis label color to white
                plot.set_ylabel('Y', color='white')  # Set Y-axis label color to white
                plot.set_title('Plot', color='white')  # Set title color to white

                # Set tick color and tick label color to white
                plot.tick_params(colors='white')
                plot.xaxis.label.set_color('white')
                plot.yaxis.label.set_color('white')


                canvas = FigureCanvas(fig)
                canvas.setGeometry(0, 0, 780, 400)
                canvas.setStyleSheet("background-color:#3533CD;")
                canvas.setParent(self.plot_space)

                self.setWindowTitle("Plot Viewer")
                self.setStyleSheet("background-color:#3533CD;")

            def refresh(self):
                for child in self.plot_space.children():
                    child.deleteLater()
                self.input_space.clear()
                self.beat_label.setText("")

            def add_index(self):
                index_text = self.input_space.text()
                if index_text:
                    self.index = int(index_text) + 1
                    self.input_space.setText(str(self.index))

            def subtract_index(self):
                index_text = self.input_space.text()
                if index_text and int(index_text) > 0:
                    self.index = int(index_text) - 1
                    self.input_space.setText(str(self.index))

            def plot_graph(self):
                for child in self.plot_space.children():
                    child.deleteLater()

                index = int(self.input_space.text())
                fig = plt.Figure(figsize=(6, 4), dpi=100, facecolor="#3533CD")
                plot = fig.add_subplot(111)
                plot.plot(test_df.iloc[index].values)
                plot.set_xlabel('X', color='white')  # Set X-axis label color to white
                plot.set_ylabel('Y', color='white')  # Set Y-axis label color to white
                plot.set_title('Plot', color='white')  # Set title color to white

                # Set tick color and tick label color to white
                plot.tick_params(colors='white')
                plot.xaxis.label.set_color('white')
                plot.yaxis.label.set_color('white')

                # Set the color of the frame and the spines to white
                plot.spines['top'].set_color('white')
                plot.spines['bottom'].set_color('white')
                plot.spines['left'].set_color('white')
                plot.spines['right'].set_color('white')

                canvas = FigureCanvas(fig)
                canvas.setGeometry(0, 0, 780, 400)
                canvas.setStyleSheet("background-color:#3533CD;")
                canvas.setParent(self.plot_space)
                canvas.show()



            # Retrieve the array element from test_df based on the input index







            def save_plot(self):
                index = int(self.input_space.text())

                x_array = test_df.iloc[index].values
                x_reshaped = x_array[:186].reshape(-1, 186, 1)
                y_predict = model.predict(x_reshaped)
                y_pred_inv = np.argmax(y_predict, axis=1)

                if 0 in y_pred_inv:
                    beat_type = "Non-ectopic beats detected"
                elif 1 in y_pred_inv:
                    beat_type = "Supraventricular ectopic beats detected"
                elif 2 in y_pred_inv:
                    beat_type = "Ventricular ectopic beats detected"
                elif 3 in y_pred_inv:
                    beat_type = "Fusion Beats detected"
                elif 4 in y_pred_inv:
                    beat_type = "Unknown Beats detected"
                else:
                    beat_type = "Beat type not detected"
                self.beat_label.setText(beat_type)
                self.beat_label.setAlignment(QtCore.Qt.AlignCenter)
                self.beat_label.setStyleSheet("background-color: black; color: white;")


            def play_audio(self):
                index = int(self.input_space.text())

                if 0 <= index < len(file_list):
                    file_name = file_list[index]
                    file_path = os.path.join(folder_path, file_name)
                    string_with_backslashes = file_path
                    string_with_forwardslashes = string_with_backslashes.replace('\\', '/')
                    print(string_with_forwardslashes)

                    #playsound(string_with_forwardslashes, alias='my_audio_alias')
                    pygame.mixer.init()
                    pygame.mixer.music.load(string_with_forwardslashes)
                    pygame.mixer.music.play()


            def show_spectrogram(self):
                class_number=0
                size=70
                min_=5
                bins=65
                for child in self.plot_space.children():
                    child.deleteLater()





                index = int(self.input_space.text())

                img = test_df.iloc[20000].values


                ###
                img=test_df.loc[test_df[187]==class_number].values
                img=img[:,min_:size]
                img_flatten=img.flatten()

                final1=np.arange(min_,size)
                for i in range (img.shape[0]-1):
                    tempo1=np.arange(min_,size)
                    final1=np.concatenate((final1, tempo1), axis=None)

                img=plt.hist(final1,img_flatten, cmap=plt.cm.jet)
                plt.show()



                """spectrogram = librosa.feature.melspectrogram(y=img, sr=0.1, n_fft=100, hop_length=256)
                fig = plt.Figure(figsize=(6, 4), dpi=100, facecolor="#3533CD")
                plot = fig.add_subplot(111)
                img = librosa.display.specshow(spectrogram,  x_axis='time', y_axis='mel',
                                               sr=0.1, ax=plot)"""
                fig = plt.Figure(figsize=(6, 4), dpi=100, facecolor="#3533CD")
                plot = fig.add_subplot(111)

                # Set axis label color and font color to white
                plot.set_title('Spectrogram', color='white')
                plot.set_xlabel('Time', color='white')
                plot.set_ylabel('Frequency', color='white')

                # Set tick color and tick label color to white
                plot.tick_params(colors='white')
                plot.xaxis.label.set_color('white')
                plot.yaxis.label.set_color('white')

                # Set the color of the frame and the spines to white
                plot.spines['top'].set_color('white')
                plot.spines['bottom'].set_color('white')
                plot.spines['left'].set_color('white')
                plot.spines['right'].set_color('white')

                canvas = FigureCanvas(img)
                canvas.setGeometry(0, 0, 780, 400)
                canvas.setStyleSheet("background-color:#3533CD;")
                canvas.setParent(self.plot_space)
                canvas.show()




             

        print(2)
        app = QtWidgets.QApplication(sys.argv)
        viewer = PlotViewer()


        viewer.show()
        print(1)
        


if __name__ == "__main__":
    app = QApplication(sys.argv)
    menu_window = MenuWindow()
    menu_window.show()
    sys.exit(app.exec_())
